In [18]:
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

  Cloning https://github.com/qubvel/segmentation_models.pytorch to /tmp/pip-req-build-c3_7wz1l
  Running command git clone -q https://github.com/qubvel/segmentation_models.pytorch /tmp/pip-req-build-c3_7wz1l
  Created wheel for segmentation-models-pytorch: filename=segmentation_models_pytorch-0.1.0-py3-none-any.whl size=53464 sha256=47b77f702e9db165bfc93f032f847cc54111ea81705ac9f4f8ddac43013d0b38
  Stored in directory: /tmp/pip-ephem-wheel-cache-v8f17t3k/wheels/53/e5/fc/18292d80d3c0f4efc96cbbb72625fdbafdca303997bacfb085
Successfully built segmentation-models-pytorch
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [19]:
import os
import json
import torch
import numpy as np
import segmentation_models_pytorch as smp
import torchvision.transforms as standard_transforms
import utils.joint_transforms as joint_transforms
import utils.transforms as extended_transforms

from datasets import mapillary
from utils import CrossEntropyLoss2d
from torch.utils.data import DataLoader

In [20]:
HOME = '/home/ec2-user/SageMaker/'

In [21]:
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
ACTIVATION = 'softmax2d'
DEVICE = 'cuda'

with open(os.path.join(HOME, 'config.json')) as config_file:
    config = json.load(config_file)

CLASSES = [] # get from config
labels = config['labels']
for label in labels:
    CLASSES.append(label['readable'])

In [22]:
# create segmentation model with pretrained encoder
model = smp.DeepLabV3(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=len(CLASSES), 
    activation=ACTIVATION,
)

model

DeepLabV3(
  (encoder): ResNetEncoder(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [23]:
args = {
    'train_batch_size': 16,
    'epoch_num': 8,
    'lr': 1e-10,
    'weight_decay': 5e-4,
    'input_size': (256, 512),
    'momentum': 0.95,
    'lr_patience': 100,  # large patience denotes fixed lr
    'snapshot': '',  # empty string denotes no snapshot
    'print_freq': 20,
    'val_batch_size': 16,
    'val_save_to_img_file': False,
    'val_img_sample_rate': 0.05  # randomly sample some validation results to display
}

In [24]:
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
short_size = int(min(args['input_size']) / 0.875)
train_joint_transform = joint_transforms.Compose([
    joint_transforms.Scale(short_size),
    joint_transforms.RandomCrop(args['input_size']),
    joint_transforms.RandomHorizontallyFlip()
])
val_joint_transform = joint_transforms.Compose([
    joint_transforms.Scale(short_size),
    joint_transforms.CenterCrop(args['input_size'])
])
input_transform = standard_transforms.Compose([
    standard_transforms.ToTensor(),
    standard_transforms.Normalize(*mean_std)
])
target_transform = extended_transforms.MaskToTensor()

In [25]:
train_set = mapillary.Mapillary('semantic', 'training', joint_transform=train_joint_transform,
                                    transform=input_transform, target_transform=target_transform)
train_loader = DataLoader(train_set, batch_size=args['train_batch_size'], num_workers=12, shuffle=True, 
                            pin_memory=True)
val_set = mapillary.Mapillary('semantic', 'validation', joint_transform=val_joint_transform, transform=input_transform,
                                target_transform=target_transform)
val_loader = DataLoader(val_set, batch_size=args['val_batch_size'], num_workers=4, shuffle=False,
                        pin_memory=True)

In [26]:
train_set[0][1].shape

torch.Size([256, 512])

In [27]:
len(CLASSES)

66

In [28]:
def iou(pred, target, n_classes=None):
    if n_classes is None:    
        assert len(pred.size()) == 4
        n_classes = pred.size()[1]

    ious = []
    pred = pred.argmax(dim=1).view(-1)
    target = target.view(-1)

    for cls in range(n_classes):  
        pred_inds = pred == cls
        target_inds = target == cls
        # Cast to long to prevent overflows
        intersection = (pred_inds[target_inds]).long().sum().data.cpu().item() 
        union = (pred_inds.long().sum().data.cpu().item() +
                 target_inds.long().sum().data.cpu().item() -
                 intersection)

        # do not include in evaluation if no gt
        if union != 0:
            ious.append(float(intersection) / float(max(union, 1)))

    return torch.Tensor(ious).mean()

def iou_mapillary(pred, targ, n_classes=len(CLASSES)):
    return iou(pred, targ, n_classes)

def acc_mapillary(pred, targ):
    targ = targ.squeeze(1)
    return (pred.argmax(dim=1)==targ).float().mean()

In [29]:
# loss = smp.utils.losses.DiceLoss()
# metrics = [
#     smp.utils.metrics.IoU(threshold=0.5),
#     smp.utils.metrics.Accuracy()
# ]

class IouMapillary(smp.utils.base.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(self, y_pr, y_gt):
        return iou_mapillary(y_pr, y_gt)

class AccuracyMapillary(smp.utils.base.Metric):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def forward(self, y_pr, y_gt):
        return acc_mapillary(y_pr, y_gt)

loss = CrossEntropyLoss2d(size_average=False).cuda()
metrics = [
    IouMapillary(),
    AccuracyMapillary()
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

In [30]:
# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)

In [31]:
loss.__name__

'CrossEntropyLoss2d'

In [32]:
!date

Sun Jun 14 11:04:27 UTC 2020


In [33]:
# train model for 8 epochs
max_score = 0
for i in range(8):
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(val_loader)
    
    print(valid_logs)
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_mapillary']:
        max_score = valid_logs['iou_mapillary']
        torch.save(model, './best_model.pth')
        print('Model saved!')


Epoch: 0
valid: 100%|██████████| 125/125 [00:23<00:00,  5.43it/s, CrossEntropyLoss2d - 8.111e+06, iou_mapillary - 0.04621, accuracy_mapillary - 0.3486]
{'CrossEntropyLoss2d': 8110994.351999998, 'iou_mapillary': 0.04621388575434685, 'accuracy_mapillary': 0.3486188316345213}
Model saved!

Epoch: 1
valid: 100%|██████████| 125/125 [00:23<00:00,  5.30it/s, CrossEntropyLoss2d - 8.104e+06, iou_mapillary - 0.04583, accuracy_mapillary - 0.3512]
{'CrossEntropyLoss2d': 8103616.595999998, 'iou_mapillary': 0.04583303195238113, 'accuracy_mapillary': 0.3511723823547363}

Epoch: 2
valid: 100%|██████████| 125/125 [00:23<00:00,  5.29it/s, CrossEntropyLoss2d - 8.181e+06, iou_mapillary - 0.05027, accuracy_mapillary - 0.316] 
{'CrossEntropyLoss2d': 8180773.412, 'iou_mapillary': 0.05026972541213036, 'accuracy_mapillary': 0.3159501571655274}
Model saved!

Epoch: 3
valid: 100%|██████████| 125/125 [00:23<00:00,  5.31it/s, CrossEntropyLoss2d - 8.106e+06, iou_mapillary - 0.06356, accuracy_mapillary - 0.3505]
{'

In [34]:
!date

Sun Jun 14 12:46:50 UTC 2020
